In [4]:
import pandas as pd
import numpy as np
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import random
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import re
from sklearn.linear_model import LogisticRegression
#import ktrain
#from ktrain import text
from joblib import dump, load
import transformers

In [ ]:
df = pd.read_csv('data/Hackathon_Base_Treino_comdep.csv')
dfsub = pd.read_csv('data/Hackathon_Base_Teste.csv')

In [ ]:
less = (df['SUB-CATEGORIA'].value_counts() < 2)
cats = []
for i in range(len(less)):
    if(less[i]):
        cats.append(less.keys()[i])
for i in range(len(df)):
    if(df['SUB-CATEGORIA'][i] in cats):
        df = df.append(df.iloc[i,:])
        
vect = TfidfVectorizer(max_features=10000).fit(x_train)

In [ ]:
cat_enc = LabelEncoder()
target_cat = cat_enc.fit_transform(df['CATEGORIA'])

x_train, x_test, y_train, y_test = train_test_split(df['DESCRIÇÃO PARCEIRO'], target_cat, test_size = 0.1, stratify = target_cat, random_state=42)

In [ ]:
t2 = text.Transformer('distilroberta-base', maxlen=100, classes = np.unique(target_cat))
train2 = t2.preprocess_train(list(x_train), y_train)
test2 = t2.preprocess_test(list(x_test), y_test)
model2 = t2.get_classifier()
learner2 = ktrain.get_learner(model2, train_data = train2, val_data=test2, batch_size=6)
learner2.model.load_weights('weights-19cat.hdf5')

In [16]:
nb = load('models/cat/nbcat.joblib'+ '.compressed') 
mlp = load('models/cat/mlpcat.joblib'+ '.compressed') 
knn = load('models/cat/knncat.joblib'+ '.compressed') 
cnb = load('models/cat/cnbcat.joblib'+ '.compressed') 
rf = load('models/cat/rfcat.joblib'+ '.compressed') 

In [17]:
rf 

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=80, n_jobs=None, oob_score=False,
                       random_state=42, verbose=0, warm_start=False)

In [ ]:
model_weights = {'BERT2': 0.8691302469936113,
 'CNB': 0.26971209717568634,
 'KNN': 0.0011019891538130155,
 'MLP': 0.17810369061951897,
 'MULTINOMIALNB': 0.7467697788345558,
 'RF': 0.6671639043827671}

model_dict = dict(zip(['MULTINOMIALNB', "BERT2", 'MLP', 'RF', 'KNN', 'CNB'],
                      [nb, learner2, mlp, rf, knn, cnb]))

In [ ]:
## Bert2
predictor = ktrain.get_predictor(learner2.model, preproc = t2)
pred3 = predictor.predict(list(aux), return_proba = True)
pred3 = pred3*model_weights['BERT2']

## Others
pred2 = 0
for model_name, model in model_dict.items():
    print(model_name)
    if((model_name != "BERT") & (model_name != "BERT2") & (model_name != "SVC")):
        pred2 += model_dict[model_name].predict_proba(vect.transform(aux)) * model_weights[model_name]
        
## Average
pred6 = (pred2+pred3)/sum(model_weights.values())
dfsub['CATEGORIA'] = cat_enc.inverse_transform(np.argmax(pred6, axis=1))
dfsub.to_csv('data/Hackathon_Base_Teste.csv', index = False)